In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Import API key
api_key = '4c40af20838b5ef25e18bea36e5b841e0ec5138d'
c = Census(api_key, year = 2013)

# Output File (CSV)
output_data_file = "output_data/census.csv"

In [7]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", 
                          "B01003_001E", 
                          "B02001_001E", 
                          "B02008_001E", 
                          "B02009_001E", 
                          "B02011_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {"for": "zip code tabulation area: 63136, 63147, 63115, 63120," +\
                                                                             "63107, 63113, 63106, 63137,"+\
                                                                             "63125, 63111, 63123, 63118,"+\
                                                                             "63112, 63119, 63116, 63143,"+\
                                                                             "63104, 63109, 63139, 63103,"+\
                                                                             "63101, 63110, 63102, 63117,"+\
                                                                             "63133, 63108, 63130, 63105"})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B02001_001E": "Race",
                                      "B02008_001E": "White", 
                                      "B02009_001E": "Black",
                                      "B02011_001E": "Asian",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "Zip Code"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zip Code", "Name", "Population", "Race", "White", "Black", "Asian", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]
census_pd

,Zip Code,Name,Population,Race,White,Black,Asian,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,63130,ZCTA5 63130,30810.0,30810.0,17389.0,12585.0,1560.0,55976.0,36054.0,4969.0,16.127881,5.066537
1,63133,ZCTA5 63133,8210.0,8210.0,558.0,7729.0,12.0,25239.0,13025.0,2815.0,34.287454,10.913520
2,63136,ZCTA5 63136,47220.0,47220.0,4834.0,42677.0,170.0,31051.0,16086.0,13019.0,27.570945,10.887336
3,63137,ZCTA5 63137,20546.0,20546.0,4458.0,16074.0,180.0,34019.0,16315.0,5275.0,25.674097,9.383822
4,63139,ZCTA5 63139,22212.0,22212.0,19132.0,2474.0,721.0,47280.0,30918.0,2667.0,12.007023,3.439582
5,63143,ZCTA5 63143,9996.0,9996.0,8165.0,1653.0,293.0,36210.0,25539.0,1905.0,19.057623,6.312525
6,63147,ZCTA5 63147,10534.0,10534.0,692.0,9800.0,20.0,29816.0,16969.0,2588.0,24.568065,10.375926
7,63101,ZCTA5 63101,2613.0,2613.0,1277.0,1284.0,62.0,54417.0,42701.0,610.0,23.344814,6.926904
8,63102,ZCTA5 63102,2216.0,2216.0,1012.0,1103.0,78.0,54018.0,32318.0,142.0,6.407942,2.572202
9,63103,ZCTA5 63103,6824.0,6824.0,3417.0,2961.0,402.0,34719.0,30274.0,1340.0,19.636577,5.216882


In [8]:
store_count = pd.read_csv(
    "zip_group.csv", dtype="object", encoding="utf-8")
store_count.head()

,Zip Code,Grocery Store Count
0,63141,5
1,63108,4
2,63118,4
3,63017,4
4,63116,3


In [9]:
df_complete = pd.merge(
    census_pd, store_count, how="left", on=["Zip Code", "Zip Code"])

In [10]:
df_complete

,Zip Code,Name,Population,Race,White,Black,Asian,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Grocery Store Count
0,63130,ZCTA5 63130,30810.0,30810.0,17389.0,12585.0,1560.0,55976.0,36054.0,4969.0,16.127881,5.066537,1
1,63133,ZCTA5 63133,8210.0,8210.0,558.0,7729.0,12.0,25239.0,13025.0,2815.0,34.287454,10.913520,NaN
2,63136,ZCTA5 63136,47220.0,47220.0,4834.0,42677.0,170.0,31051.0,16086.0,13019.0,27.570945,10.887336,NaN
3,63137,ZCTA5 63137,20546.0,20546.0,4458.0,16074.0,180.0,34019.0,16315.0,5275.0,25.674097,9.383822,NaN
4,63139,ZCTA5 63139,22212.0,22212.0,19132.0,2474.0,721.0,47280.0,30918.0,2667.0,12.007023,3.439582,2
5,63143,ZCTA5 63143,9996.0,9996.0,8165.0,1653.0,293.0,36210.0,25539.0,1905.0,19.057623,6.312525,NaN
6,63147,ZCTA5 63147,10534.0,10534.0,692.0,9800.0,20.0,29816.0,16969.0,2588.0,24.568065,10.375926,NaN
7,63101,ZCTA5 63101,2613.0,2613.0,1277.0,1284.0,62.0,54417.0,42701.0,610.0,23.344814,6.926904,1
8,63102,ZCTA5 63102,2216.0,2216.0,1012.0,1103.0,78.0,54018.0,32318.0,142.0,6.407942,2.572202,NaN
9,63103,ZCTA5 63103,6824.0,6824.0,3417.0,2961.0,402.0,34719.0,30274.0,1340.0,19.636577,5.216882,NaN
